In [33]:
# Importación liberías 
import pandas as pd
import numpy as np
import datetime
import boto3
import psycopg2
import configparser


In [39]:
rdsIdentifier = 'proyecto-db-l' #nombre de la instancia
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

In [40]:
#Conexión al Servicio de Bases de Datos Relaciones de Amazon
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-1')

In [45]:
#Declarar el nombre a la instancia en AWS
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['proyecto-db-l']


In [42]:
#Creación de la intancia en AWS
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="postgres",
            MasterUsername=config.get('RDS', 'DB_USER'),
            MasterUserPassword=config.get('RDS', 'DB_PASSWORD'),
            Port=int(config.get('RDS', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

{'DBInstance': {'DBInstanceIdentifier': 'proyecto-db-l', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'postgres', 'DBName': 'proyecto', 'AllocatedStorage': 10, 'PreferredBackupWindow': '10:16-10:46', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0cde5fe00054c4def', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres14', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0f624085bff542291', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-079951f7aaff67a36', 'SubnetAvailabilityZone': {'Name': 'us-east-1c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-04923d0f2d5f6694b', 'SubnetAvailabilityZone': {'Name': 'us-east-1f'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet

In [46]:
# Extracción del Link RDS de Postgre
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

proyecto-db-l.c7dykpfqxcl9.us-east-1.rds.amazonaws.com


In [50]:
import instruccionessql

#logueo de usuario de conexión y creacion de Tablas en SQL Postgre para el escenario
try:
    db_conn = psycopg2.connect(
        database=config.get('RDS', 'DB_NAME'), 
        user=config.get('RDS', 'DB_USER'),
        password=config.get('RDS', 'DB_PASSWORD'), 
        host=RDS_HOST,
        port=config.get('RDS', 'DB_PORT')
    )

    cursor = db_conn.cursor()
    cursor.execute(instruccionessql.DDL_QUERY)
    db_conn.commit()
    print("Base de Datos Creada Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de Datos Creada Exitosamente


In [51]:
#Alojamiento en AWS S3 para los archivos 
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-1',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)
    

bkproyecto


In [52]:
#Lectura del bucket de S3 de los 6 archivos base
S3_BUCKET_NAME = 'bkproyecto'
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['Categorias.xlsx',
 'Direcciones.xlsx',
 'Estado.xlsx',
 'MetodosPago.xlsx',
 'Pedidos.xlsx',
 'Producto.xlsx']

In [56]:
import io

#Lectura del archivo categorias. 
target_file = "Categorias.xlsx"

for remoteFile in remoteFileList:
    if remoteFile == target_file:
        try:
            file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
            data = file['Body'].read()
            df_categorias = pd.read_excel(io.BytesIO(data), engine='openpyxl')
        except Exception as ex:
            print("No es un archivo.")
            print(ex)
        break  

df_categorias.head()


,category_id,category
0,1,Men's Fashion
1,2,Women's Fashion
2,3,Appliances
3,4,Health & Sports
4,5,Kids & Baby


In [65]:
import io

#Lectura del archivo Direcciones. 
target_file = "Direcciones.xlsx"

for remoteFile in remoteFileList:
    if remoteFile == target_file:
        try:
            file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
            data = file['Body'].read()
            df_direcciones = pd.read_excel(io.BytesIO(data), engine='openpyxl')
        except Exception as ex:
            print("No es un archivo.")
            print(ex)
        break  

df_direcciones.head()

,direccion_id,place_name,county,city,state,zip,region
0,1,Brasstown,Clay,Brasstown,NC,28902,South
1,2,Pompano Beach,Broward,Pompano Beach,FL,33073,South
2,3,Roggen,Weld,Roggen,CO,80652,West
3,4,Belgrade,Kennebec,Belgrade,ME,4917,Northeast
4,5,Williamson,Lucas,Williamson,IA,50272,Midwest


In [58]:
import io

#Lectura del archivo estado. 
target_file = "Estado.xlsx"

for remoteFile in remoteFileList:
    if remoteFile == target_file:
        try:
            file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
            data = file['Body'].read()
            df_estado = pd.read_excel(io.BytesIO(data), engine='openpyxl')
        except Exception as ex:
            print("No es un archivo.")
            print(ex)
        break  

df_estado.head()

,status_id,status
0,1,complete
1,2,received
2,3,order_refunded
3,4,canceled
4,5,cod


In [74]:
import io

#Lectura del archivo Metodos de Pago.
target_file = "MetodosPago.xlsx"

for remoteFile in remoteFileList:
    if remoteFile == target_file:
        try:
            file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
            data = file['Body'].read()
            df_MetodosPago = pd.read_excel(io.BytesIO(data), engine='openpyxl')
        except Exception as ex:
            print("No es un archivo.")
            print(ex)
        break  

df_MetodosPago.head()

,payment_method_id,payment_method
0,1,cod
1,2,customercredit
2,3,jazzvoucher
3,4,easypay_voucher
4,5,bankalfalah


In [72]:
import io

#Lectura del archivo Pedidos u Ordenes es el archivo que contiene la Factable.
target_file = "Pedidos.xlsx"

for remoteFile in remoteFileList:
    if remoteFile == target_file:
        try:
            file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
            data = file['Body'].read()
            df_pedido = pd.read_excel(io.BytesIO(data), engine='openpyxl')
        except Exception as ex:
            print("No es un archivo.")
            print(ex)
        break  # Salir del bucle for una vez que se haya encontrado y leído el archivo

df_pedido.head()

,order_id,order_date,status_id,status,item_id,sku,qty_ordered,price,value,discount_amount,...,phone_no,direccion_id,place_name,county,city,state,zip,region,user_name,discount_percent
0,1001,12/20/2020,1,complete,1,MEFPAK5A094C054968C-M,1,69.9,69.9,0.0,...,252-844-9836,1,Brasstown,Clay,Brasstown,NC,28902,South,ycflanigan,0.0
1,1002,12/21/2020,1,complete,2,MEFPAK5A094BF436BE6-L,1,69.9,69.9,0.0,...,252-844-9836,1,Brasstown,Clay,Brasstown,NC,28902,South,ycflanigan,0.0
2,1003,12/21/2020,1,complete,3,MEFPAK5A094BF6361A4-M,1,69.9,69.9,0.0,...,252-844-9836,1,Brasstown,Clay,Brasstown,NC,28902,South,ycflanigan,0.0
3,1004,12/21/2020,1,complete,4,MEFPAK5A094BF6361A4-L,1,69.9,69.9,0.0,...,252-844-9836,1,Brasstown,Clay,Brasstown,NC,28902,South,ycflanigan,0.0
4,1005,12/14/2020,1,complete,5,MEFAYB59FAB695BD4A4-L,1,69.9,69.9,0.0,...,239-901-5714,2,Pompano Beach,Broward,Pompano Beach,FL,33073,South,rcoswald,0.0


In [67]:
import io

#Lectura del archivo Productos.
target_file = "Producto.xlsx"

for remoteFile in remoteFileList:
    if remoteFile == target_file:
        try:
            file = s3.Bucket(S3_BUCKET_NAME).Object(remoteFile).get()
            data = file['Body'].read()
            df_producto = pd.read_excel(io.BytesIO(data), engine='openpyxl')
        except Exception as ex:
            print("No es un archivo.")
            print(ex)
        break  # Salir del bucle for una vez que se haya encontrado y leído el archivo

df_producto.head()

,product_id,sku,category_id
0,1,MEFPAK5A094C054968C-M,1
1,2,MEFPAK5A094BF436BE6-L,1
2,3,MEFPAK5A094BF6361A4-M,1
3,4,MEFPAK5A094BF6361A4-L,1
4,5,MEFAYB59FAB695BD4A4-L,1


In [62]:
#Defición de la función que leera los datos de un DataFrame y lo enviara a un tabla de SQL Postgre
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)
          

In [66]:
#Realiza la inserción de datos en el servidor de SQL Postgre
insertDataToSQL(df_direcciones, 'addresses')

Se han insertado 773 nuevos registros.


In [68]:
insertDataToSQL(df_producto, 'products')

Se han insertado 929 nuevos registros.


In [73]:
insertDataToSQL(df_pedido, 'orders')

Se han insertado 392 nuevos registros.


In [ ]:
#Manipulacion de datos y preparacion para carga

In [75]:
#Driver para la lectura de datos en Postgre SQL
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""  

In [118]:
sql_query = 'SELECT order_date FROM orders;'
dimDate = pd.read_sql(sql_query, postgres_driver)
dimDate.head()

,order_date
0,2020-12-24
1,2020-12-24
2,2020-12-27
3,2020-12-19
4,2020-12-20


In [119]:
dimDate['year'] = pd.DatetimeIndex(dimDate['order_date']).year
dimDate['month'] = pd.DatetimeIndex(dimDate['order_date']).month
dimDate['quarter'] = pd.DatetimeIndex(dimDate['order_date']).quarter
dimDate['day'] = pd.DatetimeIndex(dimDate['order_date']).day
dimDate['week'] = pd.DatetimeIndex(dimDate['order_date']).week
dimDate['dayofweek'] = pd.DatetimeIndex(dimDate['order_date']).dayofweek
dimDate['is_weekend'] = dimDate['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dimDate.head()

C:\Users\DAVID\AppData\Local\Temp\ipykernel_8500\287956713.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimDate['week'] = pd.DatetimeIndex(dimDate['order_date']).week


,order_date,year,month,quarter,day,week,dayofweek,is_weekend
0,2020-12-24,2020,12,4,24,52,3,0
1,2020-12-24,2020,12,4,24,52,3,0
2,2020-12-27,2020,12,4,27,52,6,1
3,2020-12-19,2020,12,4,19,51,5,0
4,2020-12-20,2020,12,4,20,51,6,1


In [128]:
dimDate['order_date_id'] = dimDate['year'].astype(str) + dimDate['month'].apply(lambda x: str(x).zfill(2))
dimDate['order_date_id'] = dimDate['order_date_id'].astype(str) + dimDate['day'].apply(lambda x: str(x).zfill(2))
dimDate.drop_duplicates(inplace=True)
dimDate.head()

,order_date,year,month,quarter,day,week,dayofweek,is_weekend,order_date_id
0,2020-12-24,2020,12,4,24,52,3,0,20201224
2,2020-12-27,2020,12,4,27,52,6,1,20201227
3,2020-12-19,2020,12,4,19,51,5,0,20201219
4,2020-12-20,2020,12,4,20,51,6,1,20201220
5,2020-12-21,2020,12,4,21,52,0,0,20201221


In [87]:
df_estado.drop_duplicates(inplace=True)
df_estado.head()

,status_id,status
0,1,complete
1,2,received
2,3,order_refunded
3,4,canceled
4,5,cod


In [84]:
#Seleccionar datos de la tabla products
sql_query = 'SELECT * FROM products;'
df_SqlProduct = pd.read_sql(sql_query, postgres_driver)
df_SqlProduct.head()

,product_id,sku,category_id
0,1,MEFPAK5A094C054968C-M,1
1,2,MEFPAK5A094BF436BE6-L,1
2,3,MEFPAK5A094BF6361A4-M,1
3,4,MEFPAK5A094BF6361A4-L,1
4,5,MEFAYB59FAB695BD4A4-L,1


In [139]:
df_SqlProduct = df_SqlProduct.merge(df_categorias, how='inner', on='category_id')
df_SqlProduct = df_SqlProduct.rename(columns={'product_id': 'item_id'})
df_SqlProduct.drop_duplicates(inplace=True)
df_SqlProduct.head()

,item_id,sku,category_id,category
0,1,MEFPAK5A094C054968C-M,1,Men's Fashion
1,2,MEFPAK5A094BF436BE6-L,1,Men's Fashion
2,3,MEFPAK5A094BF6361A4-M,1,Men's Fashion
3,4,MEFPAK5A094BF6361A4-L,1,Men's Fashion
4,5,MEFAYB59FAB695BD4A4-L,1,Men's Fashion


In [88]:
df_MetodosPago.drop_duplicates(inplace=True)
df_MetodosPago.head()

,payment_method_id,payment_method
0,1,cod
1,2,customercredit
2,3,jazzvoucher
3,4,easypay_voucher
4,5,bankalfalah


In [89]:
#Seleccionar datos de la tabla orders
sql_query = 'SELECT * FROM orders;'
df_SqlCust = pd.read_sql(sql_query, postgres_driver)
df_SqlCust.head()

,order_id,order_date,status_id,status,item_id,sku,qty_ordered,price,value,discount_amount,...,phone_no,direccion_id,place_name,county,city,state,zip,region,user_name,discount_percent
0,2784,2020-12-24,1,complete,159,MEFPAK5A094C08D9385-L,1,29.9,29.9,0.0,...,308-477-6020,400,Long Pine,Brown,Long Pine,NE,69217,Midwest,dpcrank,0.0
1,2785,2020-12-24,1,complete,159,MEFPAK5A094C08D9385-L,1,29.9,29.9,0.0,...,304-754-1135,894,Thorpe,McDowell,Thorpe,WV,24888,South,jituma,0.0
2,2786,2020-12-27,1,complete,95,MEFPAK5A094C08D9385-M,1,29.9,29.9,0.0,...,262-259-6198,1274,Amherst,Portage,Amherst,WI,54406,Midwest,fdbarela,0.0
3,2787,2020-12-19,1,complete,162,MEFPAK5A094C0B66DD1-L,1,29.9,29.9,0.0,...,252-261-2085,912,Fayetteville,Cumberland,Fayetteville,NC,28306,South,bpdreiling,0.0
4,2788,2020-12-20,1,complete,95,MEFPAK5A094C08D9385-M,1,29.9,29.9,0.0,...,307-852-5116,1275,Jelm,Albany,Jelm,WY,82063,West,dusymons,0.0


In [90]:
selected_columns = ['cust_id', 'ref_num', 'name_prefix','first_name','middle_initial','last_name','gender','age','full_name','email','customer_since' ,'ssn','phone_no','user_name']
df_SqlCust = df_SqlCust[selected_columns]
df_SqlCust.drop_duplicates(inplace=True)
df_SqlCust.head()

,cust_id,ref_num,name_prefix,first_name,middle_initial,last_name,gender,age,full_name,email,customer_since,ssn,phone_no,user_name
0,8793,858912,Dr.,Dale,P,Crank,M,73,"Crank, Dale",dale.crank@hotmail.com,2017-05-04,159-86-7037,308-477-6020,dpcrank
1,64908,390224,Mr.,Jean,I,Tuma,M,39,"Tuma, Jean",jean.tuma@earthlink.net,2016-04-02,127-98-1999,304-754-1135,jituma
2,65494,926974,Mr.,Frank,D,Barela,M,29,"Barela, Frank",frank.barela@shell.com,2015-07-29,311-35-4632,262-259-6198,fdbarela
3,67329,510421,Mr.,Basil,P,Dreiling,M,58,"Dreiling, Basil",basil.dreiling@gmail.com,2012-10-15,141-23-0498,252-261-2085,bpdreiling
4,67508,200615,Hon.,Dylan,U,Symons,M,19,"Symons, Dylan",dylan.symons@aol.com,1998-11-19,307-37-7976,307-852-5116,dusymons


In [92]:
#Seleccionar datos de la tabla addresses
sql_query = 'SELECT * FROM addresses;'
df_SqlAddress = pd.read_sql(sql_query, postgres_driver)
df_SqlAddress.drop_duplicates(inplace=True)
df_SqlAddress.head()

,direccion_id,place_name,county,city,state,zip,region
0,1,Brasstown,Clay,Brasstown,NC,28902,South
1,2,Pompano Beach,Broward,Pompano Beach,FL,33073,South
2,3,Roggen,Weld,Roggen,CO,80652,West
3,4,Belgrade,Kennebec,Belgrade,ME,4917,Northeast
4,5,Williamson,Lucas,Williamson,IA,50272,Midwest


In [132]:
#Seleccionar datos de la tabla orders
sql_query = 'SELECT * FROM orders;'
df_SqlFactable = pd.read_sql(sql_query, postgres_driver)
df_SqlFactable.head()

,order_id,order_date,status_id,status,item_id,sku,qty_ordered,price,value,discount_amount,...,phone_no,direccion_id,place_name,county,city,state,zip,region,user_name,discount_percent
0,2784,2020-12-24,1,complete,159,MEFPAK5A094C08D9385-L,1,29.9,29.9,0.0,...,308-477-6020,400,Long Pine,Brown,Long Pine,NE,69217,Midwest,dpcrank,0.0
1,2785,2020-12-24,1,complete,159,MEFPAK5A094C08D9385-L,1,29.9,29.9,0.0,...,304-754-1135,894,Thorpe,McDowell,Thorpe,WV,24888,South,jituma,0.0
2,2786,2020-12-27,1,complete,95,MEFPAK5A094C08D9385-M,1,29.9,29.9,0.0,...,262-259-6198,1274,Amherst,Portage,Amherst,WI,54406,Midwest,fdbarela,0.0
3,2787,2020-12-19,1,complete,162,MEFPAK5A094C0B66DD1-L,1,29.9,29.9,0.0,...,252-261-2085,912,Fayetteville,Cumberland,Fayetteville,NC,28306,South,bpdreiling,0.0
4,2788,2020-12-20,1,complete,95,MEFPAK5A094C08D9385-M,1,29.9,29.9,0.0,...,307-852-5116,1275,Jelm,Albany,Jelm,WY,82063,West,dusymons,0.0


In [133]:
#Agregamos la Id de dates
df_SqlFactable['year'] = pd.DatetimeIndex(df_SqlFactable['order_date']).year
df_SqlFactable['month'] = pd.DatetimeIndex(df_SqlFactable['order_date']).month
df_SqlFactable['day'] = pd.DatetimeIndex(df_SqlFactable['order_date']).day

df_SqlFactable['order_date_id'] = df_SqlFactable['year'].astype(str) + df_SqlFactable['month'].apply(lambda x: str(x).zfill(2))
df_SqlFactable['order_date_id'] = df_SqlFactable['order_date_id'].astype(str) + df_SqlFactable['day'].apply(lambda x: str(x).zfill(2))

df_SqlFactable.drop_duplicates(inplace=True)
df_SqlFactable.head()

,order_id,order_date,status_id,status,item_id,sku,qty_ordered,price,value,discount_amount,...,city,state,zip,region,user_name,discount_percent,year,month,day,order_date_id
0,2784,2020-12-24,1,complete,159,MEFPAK5A094C08D9385-L,1,29.9,29.9,0.0,...,Long Pine,NE,69217,Midwest,dpcrank,0.0,2020,12,24,20201224
1,2785,2020-12-24,1,complete,159,MEFPAK5A094C08D9385-L,1,29.9,29.9,0.0,...,Thorpe,WV,24888,South,jituma,0.0,2020,12,24,20201224
2,2786,2020-12-27,1,complete,95,MEFPAK5A094C08D9385-M,1,29.9,29.9,0.0,...,Amherst,WI,54406,Midwest,fdbarela,0.0,2020,12,27,20201227
3,2787,2020-12-19,1,complete,162,MEFPAK5A094C0B66DD1-L,1,29.9,29.9,0.0,...,Fayetteville,NC,28306,South,bpdreiling,0.0,2020,12,19,20201219
4,2788,2020-12-20,1,complete,95,MEFPAK5A094C08D9385-M,1,29.9,29.9,0.0,...,Jelm,WY,82063,West,dusymons,0.0,2020,12,20,20201220


In [134]:
#Seleccionamos las columnas que necesitamos para la Factable con comandos de Panda
selected_columns = ['order_id', 'qty_ordered', 'price','value','discount_amount','total','discount_percent','order_date_id','order_date','status_id','item_id' ,'payment_method_id','cust_id','direccion_id']
df_SqlFactable = df_SqlFactable[selected_columns]
df_SqlFactable.drop_duplicates(inplace=True)
df_SqlFactable.head()

,order_id,qty_ordered,price,value,discount_amount,total,discount_percent,order_date_id,order_date,status_id,item_id,payment_method_id,cust_id,direccion_id
0,2784,1,29.9,29.9,0.0,29.9,0.0,20201224,2020-12-24,1,159,1,8793,400
1,2785,1,29.9,29.9,0.0,29.9,0.0,20201224,2020-12-24,1,159,1,64908,894
2,2786,1,29.9,29.9,0.0,29.9,0.0,20201227,2020-12-27,1,95,1,65494,1274
3,2787,1,29.9,29.9,0.0,29.9,0.0,20201219,2020-12-19,1,162,1,67329,912
4,2788,1,29.9,29.9,0.0,29.9,0.0,20201220,2020-12-20,1,95,1,67508,1275


Preparacion DW

In [82]:
rdsIdentifier = 'dw-db'

In [83]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_MYSQL', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="mysql",
            MasterUsername=config.get('RDS_MYSQL', 'DB_USER'),
            MasterUserPassword=config.get('RDS_MYSQL', 'DB_PASSWORD'),
            Port=int(config.get('RDS_MYSQL', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

{'DBInstance': {'DBInstanceIdentifier': 'dw-db', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'creating', 'MasterUsername': 'root', 'DBName': 'dw', 'AllocatedStorage': 10, 'PreferredBackupWindow': '03:34-04:04', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0cde5fe00054c4def', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-0f624085bff542291', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-079951f7aaff67a36', 'SubnetAvailabilityZone': {'Name': 'us-east-1c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-04923d0f2d5f6694b', 'SubnetAvailabilityZone': {'Name': 'us-east-1f'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-03c896dfc011a24f1', 'S

In [102]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_DW_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_DW_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

dw-db.c7dykpfqxcl9.us-east-1.rds.amazonaws.com


In [129]:
import CreacionDWquery
import mysql.connector as mysqlC
try:
    myDw = mysqlC.connect(
    host=RDS_DW_HOST, 
    user=config.get('RDS_MYSQL', 'DB_USER'),
    password=config.get('RDS_MYSQL', 'DB_PASSWORD'),
    database=config.get('RDS_MYSQL', 'DB_NAME')
    )

    mycursor = myDw.cursor()
    mycursor.execute(CreacionDWquery.CREATE_DW, multi=True)
    myDw.commit()
    print("Data Warehouse Creado Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)



ERROR: Error al crear la base de datos.
Commands out of sync; you can't run this command now


In [122]:
mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL', 'DB_USER')}:{config.get('RDS_MYSQL', 'DB_PASSWORD')}@{RDS_DW_HOST}:{config.get('RDS_MYSQL', 'DB_PORT')}/{config.get('RDS_MYSQL', 'DB_NAME')}"""  

In [131]:
#insertamos en dimDates
dimDate.to_sql('dimDate', mysql_driver, index=False, if_exists='append')

365

In [137]:
#insertamos en dimStatus
df_estado.to_sql('dimStatus', mysql_driver, index=False, if_exists='append')

13

In [140]:
#insertamos en dimProduct
df_SqlProduct.to_sql('dimProduct', mysql_driver, index=False, if_exists='append')

47929

In [141]:
#insertamos en dimPaymentMethod
df_MetodosPago.to_sql('dimPaymentMethod', mysql_driver, index=False, if_exists='append')

13

In [142]:
#insertamos en dimCust
df_SqlCust.to_sql('dimCust', mysql_driver, index=False, if_exists='append')

64248

In [143]:
#insertamos en dimAddress
df_SqlAddress.to_sql('dimAddress', mysql_driver, index=False, if_exists='append')

33773

In [144]:
#insertamos en Factable
df_SqlFactable.to_sql('Factable', mysql_driver, index=False, if_exists='append')

286392